In [1]:
import pickle, os
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid, Bar, Scatter,Page
import pandas as pd
import akshare as ak
from log import logger
from tools import *

import pandas as pd
import numpy as np
import fetch_data
from pyecharts.commons.utils import JsCode

# # 读取temp_data/stock_data_20210701_20230328.pkl
# with open('../temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
#     stock_data = pickle.load(f)


In [12]:
def make_point_bb(df, report_df):


    MarkPointItems = []
    for date in report_df.index.strftime('%Y-%m-%d').tolist():
        MarkPointItems.append(
            opts.MarkPointItem(coord=[date, df.loc[date, 'high'] * 1.02],
                            #    name=report_df.loc[date, 'report_date'] + '.<br>' + report_df.loc[date, '所处行业'],
                               name =JsCode(
                        """
                        function(params) {
                            return 'Mxxx<br>Value';
                        }
                        """
                    ),
                               itemstyle_opts=opts.ItemStyleOpts(color="blue",
                                                                 opacity=0.5)))
    return MarkPointItems


def kline_compare_base(symbol,
                       start_date,
                       end_date,
                       period='daily',
                       adjust='',
                       axis_index=0,
                       is_all_trade_dates=True):
    # 将数据转换为列表格式

    stock_df = fetch_data.fetch_hist(
        symbol=symbol,start_date=start_date, end_date=end_date, period=period, adjust=adjust)

    if is_all_trade_dates:
        stock_df = align_stock_data_to_trading_calendar(
            stock_df, start_date, end_date)
        report_df = align_reprot_data_to_trading_calendar(symbol,
                                              start_date=start_date,
                                              end_date=end_date,
                                              reports_dump='reports-2015-2023')
    else:
        report_df = align_reprot_data_to_trading_calendar(
            symbol, dates=stock_df['date'].to_list())


    data = stock_df[['open', 'close', 'low', 'high']].values.tolist()
    dates = stock_df.index.strftime('%Y-%m-%d').tolist()

    MarkPointItems = make_point_bb(stock_df, report_df)

    kline = (
        Kline(init_opts=opts.InitOpts(
            bg_color="#000000")).add_xaxis(dates).add_yaxis(
                series_name="K线图",
                y_axis=data,
                xaxis_index=axis_index,
                yaxis_index=axis_index,
                itemstyle_opts=opts.ItemStyleOpts(
                    color="#ec0000",
                    color0="#00da3c",
                    border_color="#8A0000",
                    border_color0="#008F28",
                ),
            ).set_global_opts(
                title_opts=opts.TitleOpts(title="双品种组合"),
                xaxis_opts=opts.AxisOpts(
                    # type_='category',
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=False)),
                yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=False)),
                legend_opts=opts.LegendOpts(
                    is_show=True,
                    textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))).
        set_series_opts(
            # label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(data=MarkPointItems,
                                              symbol_size=20),
        ))

    # 绘制均线图
    # 计算均线数据
    ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    ma20 = stock_df['close'].rolling(window=20).mean().tolist()

    line = (
        Line().set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
        ).add_xaxis(dates).add_yaxis("MA5", ma5,
                                     symbol='none')  # symbol='none'去掉line上的圆点
        .add_yaxis("MA10", ma10, symbol='none').add_yaxis("MA20",
                                                          ma20,
                                                          symbol='none'))

    # 将 K 线图和均线图组合在一起
    k_ma = kline.overlap(line)
    return k_ma


def stock_compare_vol(stock_df, axis_index=0):
    data = stock_df['vol'].astype(
        int).values.tolist()  # pyecharts的数据类型必须是int,如果是float,则会无法显示
    dates = stock_df.index.tolist()  # date type:str, like '2021-07-01'
    bar = (
        Bar().set_global_opts(
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False)),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(
                is_show=False,
                textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))).
        add_xaxis(dates).add_yaxis(
            series_name="成交量",
            y_axis=data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                # color0="#00da3c",
                # border_color="#8A0000",
                # border_color0="#008F28",
            ),
        ).set_series_opts(label_opts=opts.LabelOpts(is_show=False)))

    # 绘制均线图
    # 计算均线数据
    # ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    # ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    # ma20 = stock_df['close'].rolling(window=20).mean().tolist()

    # line = (
    #     Line()
    #     .set_series_opts(
    #         label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
    #     )
    #     .add_xaxis(dates)
    #     .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
    #     .add_yaxis("MA10", ma10, symbol='none')
    #     .add_yaxis("MA20", ma20, symbol='none')
    # )

    # 将 K 线图和均线图组合在一起
    # k_ma  = bar.overlap(line)
    return bar


# def line_base(start_value) -> Line:
#     stock_a_lg_indicator_df = ak.stock_a_lg_indicator(symbol="600519")
#     stock_a_lg_indicator_df = stock_a_lg_indicator_df[
#         stock_a_lg_indicator_df['trade_date'].apply(
#             lambda d: pd.Timestamp(d)) >= pd.Timestamp(start_value)]

#     line = (Line().add_xaxis(
#         xaxis_data=stock_a_lg_indicator_df.trade_date.tolist()).add_yaxis(
#             yaxis_index=0,
#             series_name="pe",
#             y_axis=stock_a_lg_indicator_df.pe.rolling(
#                 window=5).mean().tolist()).set_global_opts())
#     return line


kline = kline_compare_base('600519', '20200806', '20220329')
kline.render_notebook()


In [7]:
def kline_compare(df_top, df_bottom, start_value, end_value):

    kline_top = kline_compare_base('600519', axis_index=0)
    kline_bottom = kline_compare_base(df_bottom, axis_index=1)
    kline_top_vol = stock_compare_vol(df_top, axis_index=2)
    kline_top_bottom = stock_compare_vol(df_bottom, axis_index=3)
    line_1 = line_base(start_value)

    kline_top.set_global_opts(
        datazoom_opts=[
            opts.DataZoomOpts(is_show=False,
                              start_value=start_value,
                              end_value=end_value,
                              range_start=None,
                              range_end=None,
                              xaxis_index=[0, 0],
                              pos_top="97%",
                              pos_bottom="1%"),
            opts.DataZoomOpts(is_show=True,
                              start_value=start_value,
                              end_value=end_value,
                              range_start=None,
                              range_end=None,
                              xaxis_index=[0, 1],
                              pos_top="97%",
                              pos_bottom="1%"),
            opts.DataZoomOpts(is_show=False,
                              start_value=start_value,
                              end_value=end_value,
                              range_start=None,
                              range_end=None,
                              xaxis_index=[0, 2],
                              pos_top="97%",
                              pos_bottom="1%"),
            opts.DataZoomOpts(is_show=False,
                              start_value=start_value,
                              end_value=end_value,
                              range_start=None,
                              range_end=None,
                              xaxis_index=[0, 3],
                              pos_top="97%",
                              pos_bottom="1%"),
        ],
        # xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
    )

    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))

    grid.add(kline_top,
             grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))
    grid.add(kline_top_vol,
             grid_opts=opts.GridOpts(pos_top="40%", pos_bottom="50%"))
    grid.add(kline_bottom,
             grid_opts=opts.GridOpts(pos_top="50%", pos_bottom="10%"))
    grid.add(kline_top_bottom,
             grid_opts=opts.GridOpts(pos_top="90%", pos_bottom="0%"))
    # grid.add(line_1, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))

    return grid


df_top = stock_data[('600519', '贵州茅台')]
df_bottom = stock_data[('688766', '普冉股份')]
# df_top=get_data_from_all('600519',stock_data)
# df_bottom = stock_data[('zs.000001', '上证指数')]
gg = kline_compare(df_top, df_bottom, '2021-08-06', '2022-03-29')
kline_compare(df_top, df_bottom, '2021-08-06', '2022-03-29').render_notebook()


NameError: name 'stock_data' is not defined

In [ ]:
def kline_compare(df_top,df_bottom,start_value,end_value):

    kline_top = kline_compare_base(df_top,axis_index=0)
    kline_bottom = kline_compare_base(df_bottom,axis_index=1)
    kline_top_vol = stock_compare_vol(df_top,axis_index=2)
    kline_top_bottom = stock_compare_vol(df_bottom,axis_index=3)
    line_1 = line_base(start_value)

    kline_top.set_global_opts(datazoom_opts=[
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,0], pos_top="97%",pos_bottom="1%"),
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,1], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,2], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,3], pos_top="97%", pos_bottom="1%"),
        ],
        # xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
    )

    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))

    grid.add(kline_top, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))
    grid.add(kline_top_vol, grid_opts=opts.GridOpts(pos_top="40%", pos_bottom="50%"))
    grid.add(kline_bottom, grid_opts=opts.GridOpts(pos_top="50%", pos_bottom="10%"))
    grid.add(kline_top_bottom, grid_opts=opts.GridOpts(pos_top="90%", pos_bottom="0%"))
    # grid.add(line_1, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))

    return grid


df_top=stock_data[('600519', '贵州茅台')]
df_bottom=stock_data[('688766', '普冉股份')]
# df_top=get_data_from_all('600519',stock_data)
# df_bottom = stock_data[('zs.000001', '上证指数')]
gg = kline_compare(df_top, df_bottom, '2021-08-06', '2022-03-29')
kline_compare(df_top,df_bottom,'2021-08-06','2022-03-29').render_notebook()

In [ ]:
x_data = ['Apple', 'Huawei', 'Xiaomi', 'Oppo', 'Vivo', 'Meizu']
y_data = [123, 153, 89, 0, 98, 23]

scatter = (Scatter().add_xaxis(x_data).add_yaxis('', y_data))

page = Page(layout=Page.SimplePageLayout)
page.add(
    gg,
    scatter,
    # tab
)
page.render_notebook()

In [ ]:
from tools import tool_trade_date_hist
import pickle
import pandas as pd
import numpy as np

dates = tool_trade_date_hist('2018-08-06','2022-03-29')


def report_data_on_trade_date(stock_code,start_date = None,end_date=None,dates = None):

    if (start_date is None or end_date is None) and dates is None:
        raise Exception('start_date and end_date or dates must be provided')
        
    with open('../temp_data/all_yjbb.pkl', 'rb') as f:
        all_reports = pickle.load(f)
    report_data = all_reports[all_reports['股票代码']==stock_code].sort_values(by='最新公告日期',ascending=True)
    report_data['最新公告日期'] = pd.to_datetime(report_data['最新公告日期'])
    report_data.drop_duplicates(subset=['最新公告日期'], keep='last', inplace=True)

    trade_dates = dates if dates is not None else tool_trade_date_hist(start_date,end_date)
    trade_dates['trade_date'] = pd.to_datetime(trade_dates['trade_date'])
    report_data_on_trade_date_df = pd.merge_asof(report_data, trade_dates, left_on='最新公告日期', right_on='trade_date',direction='backward')
    
    report_data_on_trade_date_df.rename(columns={'trade_date':'date'},inplace=True)
    report_data_on_trade_date_df.rename(columns={'最新公告日期':'reprot_date'},inplace=True)
    
    return report_data_on_trade_date_df[report_data_on_trade_date_df['date'].notnull()]


# report_data_on_trade_date('600519',dates=dates)
# report_data_on_trade_date('600519')
# report_data_on_trade_date('600519',start_date = '2020-08-06',end_date='2022-03-29')
report_data_on_trade_date('600519',start_date = '2020-08-06',end_date='2022-03-29',dates=dates)




In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
import numpy as np

"""
Gallery 使用 pyecharts 1.0.0
参考地址: https://echarts.apache.org/examples/editor.html?c=multiple-y-axis

目前无法实现的功能:

1、暂无
"""

colors = ["#5793f3", "#d14a61", "#675bba"]
x_data = bb_df['date'].tolist()
legend_list = ["蒸发量", "降水量", "平均温度"]
evaporation_capacity = np.nan_to_num(bb_df['净利润-净利润']).astype(int).tolist()
rainfall_capacity = np.nan_to_num(bb_df['营业收入-季度环比增长']).astype(int).tolist()
average_temperature = np.nan_to_num(bb_df['净利润-同比增长']).astype(int).tolist()

bar = (
    Bar(init_opts=opts.InitOpts(width="1260px", height="720px"))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="蒸发量", y_axis=evaporation_capacity, yaxis_index=0, color=colors[1]
    )
    .add_yaxis(
        series_name="降水量", y_axis=rainfall_capacity, yaxis_index=1, color=colors[0]
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="蒸发量",
            type_="value",
            min_=0,
            max_=250,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[1])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="温度",
            min_=0,
            max_=25,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[2])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="降水量",
            min_=0,
            max_=250,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[0])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger="axis", axis_pointer_type="cross"),
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="平均温度", y_axis=average_temperature, yaxis_index=2, color=colors[2]
    )
)

bar.overlap(line).render_notebook()


In [ ]:
import numpy as np
x_data = bb_df['date'].tolist()
# y_data = np.nan_to_num(bb_df['净资产收益率']).astype(int).tolist()
y_data = np.nan_to_num(bb_df['营业收入-季度环比增长']).astype(int).tolist()
c = (
    Bar()
    .add_xaxis(x_data)
    .add_yaxis("商家A", y_data, bar_min_width=10,bar_min_height=0)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
        is_scale=True,
        ),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            # min_=0.001
        )
    )
    .render_notebook()
)
c

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
from pyecharts.faker import Faker


def bar_datazoom_slider() -> Bar:
    c = (Bar().add_xaxis(Faker.days_attrs).add_yaxis(
        "商家A", Faker.days_values).set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider-水平）"),
            datazoom_opts=[opts.DataZoomOpts()],
        ))
    return c


def line_markpoint() -> Line:
    c = (Line().add_xaxis(Faker.choose()).add_yaxis(
        "商家A",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="min")]),
    ).add_yaxis(
        "商家B",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="max")]),
    ).set_global_opts(title_opts=opts.TitleOpts(title="Line-MarkPoint")))
    return c


def pie_rosetype() -> Pie:
    v = Faker.choose()
    c = (Pie().add(
        "",
        [list(z) for z in zip(v, Faker.values())],
        radius=["30%", "75%"],
        center=["25%", "50%"],
        rosetype="radius",
        label_opts=opts.LabelOpts(is_show=False),
    ).add(
        "",
        [list(z) for z in zip(v, Faker.values())],
        radius=["30%", "75%"],
        center=["75%", "50%"],
        rosetype="area",
    ).set_global_opts(title_opts=opts.TitleOpts(title="Pie-玫瑰图示例")))
    return c


def grid_mutil_yaxis() -> Grid:
    x_data = ["{}月".format(i) for i in range(1, 13)]
    bar = (Bar().add_xaxis(x_data).add_yaxis(
        "蒸发量",
        [2.0, 4.9, 7.0, 23.2, 25.6, 76.7, 135.6, 162.2, 32.6, 20.0, 6.4, 3.3],
        yaxis_index=0,
        color="#d14a61",
    ).add_yaxis(
        "降水量",
        [2.6, 5.9, 9.0, 26.4, 28.7, 70.7, 175.6, 182.2, 48.7, 18.8, 6.0, 2.3],
        yaxis_index=1,
        color="#5793f3",
    ).extend_axis(yaxis=opts.AxisOpts(
        name="蒸发量",
        type_="value",
        min_=0,
        max_=250,
        position="right",
        axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
            color="#d14a61")),
        axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
    )).extend_axis(yaxis=opts.AxisOpts(
        type_="value",
        name="温度",
        min_=0,
        max_=25,
        position="left",
        axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
            color="#675bba")),
        axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
        splitline_opts=opts.SplitLineOpts(
            is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)),
    )).set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="降水量",
            min_=0,
            max_=250,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
                color="#5793f3")),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        title_opts=opts.TitleOpts(title="Grid-多 Y 轴示例"),
        tooltip_opts=opts.TooltipOpts(trigger="axis",
                                      axis_pointer_type="cross"),
    ))

    line = (Line().add_xaxis(x_data).add_yaxis(
        "平均温度",
        [2.0, 2.2, 3.3, 4.5, 6.3, 10.2, 20.3, 23.4, 23.0, 16.5, 12.0, 6.2],
        yaxis_index=2,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    ))

    bar.overlap(line)
    return Grid().add(bar,
                      opts.GridOpts(pos_left="5%", pos_right="20%"),
                      is_control_axis_index=True)


tab = Tab()
tab.add(bar_datazoom_slider(), "bar-example")
tab.add(line_markpoint(), "line-example")
tab.add(pie_rosetype(), "pie-example")
tab.add(grid_mutil_yaxis(), "grid-example")
tab.render_notebook()

In [ ]:
from tools import tool_trade_date_hist
from fetch_stock_data import fetch_one_stock_hist_from_ak
from chart_componets import kline

start_date = '20140101'
end_date= '20140630'
symbol = '000001'
period = 'daily'

stock_info = (('000001','深圳银行'),period,start_date,end_date,'')
index_info = (('zs.000001','上证指数'),period,start_date,end_date)


stock_data = fetch_one_stock_hist_from_ak(stock_info)

#删除stock_data中的10到20行
stock_data = stock_data.drop(stock_data.index[0:20])
stock_data = stock_data.drop(stock_data.index[40:45])

from tools import tool_data_cleaning_by_trade_date_hist

df = tool_data_cleaning_by_trade_date_hist(stock_data, '20140101', '20140630')

kline(df, date_name='trade_date').render_notebook()


In [ ]:
x_data = ['Apple', 'Huawei', 'Xiaomi', 'Oppo', 'Vivo', 'Meizu']
y_data = [123, 153, 89, 107, 98, 23]

# 三种定位方式如下
bar = (
    Bar().add_xaxis(x_data).add_yaxis('', y_data).set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(data=[
            # 根据坐标定位
            opts.MarkPointItem(coord=['Xiaomi', 150], name="坐标"),
            # 根据像素值定位
            opts.MarkPointItem(x=200, y=160, name="像素值"),
            # 设置显示的value
            opts.MarkPointItem(coord=[4, 150], name="设置value", value='hi'),
        ])))

bar.render_notebook()


In [15]:
all = fetch_data.get_hist('reports-2015-2023')
all[all['股票代码']=='600519']

,序号,股票代码,股票简称,每股收益,营业收入-营业收入,营业收入-同比增长,营业收入-季度环比增长,净利润-净利润,净利润-同比增长,净利润-季度环比增长,每股净资产,净资产收益率,每股经营现金流量,销售毛利率,所处行业,最新公告日期
625,626,600519,贵州茅台,9.09,2.373432e+10,7.040289,1.6617,1.142464e+10,6.838923,0.3711,47.642248,19.69,9.093626,92.452930,酿酒行业,2016-10-29
2742,2743,600519,贵州茅台,3.47,8.760368e+09,15.545618,-12.7797,4.364902e+09,17.989888,-6.2617,50.600422,7.85,1.849069,93.214204,酿酒行业,2016-04-21
2597,2598,600519,贵州茅台,3.89,1.025087e+10,17.014122,5.5426,4.889272e+09,12.013326,19.8806,54.783385,7.37,5.919485,92.341367,酿酒行业,2017-04-25
2558,2559,600519,贵州茅台,9.92,2.753274e+10,16.003897,3.6335,1.246558e+10,9.111361,-6.3992,54.643277,18.45,25.899848,91.643856,酿酒行业,2017-10-26
2158,2159,600519,贵州茅台,15.91,4.448737e+10,61.579931,64.0128,1.998385e+10,60.312240,70.3086,67.151990,24.97,18.139474,89.933118,酿酒行业,2018-10-29
540,541,600519,贵州茅台,4.87,1.391341e+10,35.729110,10.2284,6.123119e+09,25.235819,43.9790,62.902330,8.06,4.863068,91.156138,酿酒行业,2018-04-28
1122,1123,600519,贵州茅台,6.28,1.618565e+10,10.742002,-15.2400,7.888232e+09,9.106386,-19.2804,53.681658,13.75,4.292204,92.611040,酿酒行业,2016-08-27
3467,3468,600519,贵州茅台,6.77,1.839526e+10,32.212413,10.9794,8.506907e+09,38.930931,19.8914,79.571822,8.89,3.928920,91.305517,酿酒行业,2019-04-25
9067,9068,600519,贵州茅台,7.01,1.873762e+10,15.766838,-17.2095,8.802637e+09,11.592012,-19.9602,51.726656,12.88,10.829475,91.881425,酿酒行业,2017-07-28
6577,6578,600519,贵州茅台,12.34,3.344686e+10,3.816734,28.6656,1.550309e+10,0.998618,15.3276,50.888466,26.23,13.880251,92.227894,酿酒行业,2017-04-15


In [15]:
import akshare as ak

stock_financial_analysis_indicator_df = ak.stock_financial_analysis_indicator(symbol="600519")
print(stock_financial_analysis_indicator_df)
# 用pickle 存储stock_financial_analysis_indicator_df 
   



            日期 摊薄每股收益(元) 加权每股收益(元) 每股收益_调整后(元) 扣除非经常性损益后的每股收益(元) 每股净资产_调整前(元)  \
0   2022-12-31   52.0421     49.93       49.93             49.99     163.1628   
1   2022-09-30   36.8684     35.34       35.34                --     169.9148   
2   2022-06-30   24.7615     23.72       23.72             23.69     157.9818   
3   2022-03-31   14.2908     13.73       13.73                --     171.0784   
4   2021-12-31   44.3565     41.76       41.76             41.86     156.7886   
..         ...       ...       ...         ...               ...          ...   
84  2001-12-31    1.3695        --          --                --      10.1505   
85  2001-06-30      1.21        --          --              1.21           --   
86  2000-12-31      1.36        --          --              1.36           --   
87  1999-12-31    1.1658        --          --                --           --   
88  1998-12-31        --        --          --                --           --   

   每股净资产_调整后(元) 每股经营性现金流(元)

In [17]:
stock_financial_analysis_indicator_df[['日期','净利润增长率(%)']][:]

,日期,净利润增长率(%)
0,2022-12-31,17.3268
1,2022-09-30,17.0285
2,2022-06-30,18.6925
3,2022-03-31,21.5278
4,2021-12-31,12.5137
...,...,...
84,2001-12-31,33.3047
85,2001-06-30,--
86,2000-12-31,19.0734
87,1999-12-31,46.8364


In [ ]:
import akshare as ak

stock_yjbb_em_df = ak.stock_yjbb_em(date="20220331")
print(stock_yjbb_em_df)
